# Collision Avoidance - collecte button
Ce notebook permet de réaliser les photos des différents panneaux, objets et situations (voie libre, piéton), puis de les classer dans différents dossiers afin de les utiliser pour réaliser le modèle (dans train modèle).

### Importation des bibliothèques et constantes

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
import os

filenames = ['blocked','free','stopsign','vitesses','directions','pieton']
vitesses = ['speed1sign','speed2sign','speed3sign']
directions = ['moveright','moveleft']
filenames.sort()
vitesses.sort()
directions.sort()

largeur_image = 600
hauteur_image = 600

### Initialisation de la caméra et création des différents dossiers

In [ ]:
camera = Camera.instance(width=largeur_image, height=hauteur_image,fps=5) # nous réglons le nombre d'images par seconde à 5 afin de limiter le flux de données et garder une visualisation non décalée, même si elle est sacadée
image = widgets.Image(format='jpeg', width=largeur_image, height=hauteur_image)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

#---- CREATION DES DOSSIERS CONTENANT LES DIFFERENTES PHOTOS -----

detectnames = filenames + vitesses + directions

detectdirs = []
for i in range(len(filenames)):
    detectdirs.append('dataset/'+ filenames[i])
try:
    for i in range(len(detectdirs)):
        os.makedirs(detectdirs[i])
except FileExistsError:
    print('Directories not created because they already exist')
    
    
detectdirs_cropped = []
for i in range(len(vitesses)):
    detectdirs_cropped.append('dataset_cropped/'+ vitesses[i])
for i in range(len(directions)):
    detectdirs_cropped.append('dataset_cropped/'+ directions[i])
try:
    for i in range(len(detectdirs_cropped)):
        os.makedirs(detectdirs_cropped[i])
except FileExistsError:
    print('Directories not created because they already exist')


### Création des boutons et compteurs

In [ ]:
#FORME ET DIMENSIONS DES BOUTONS
button_layout = widgets.Layout(width='200px', height='32px')

#BOUTONS___________________________
#généraux
#Pour modifier l'apparence on utilise: button_style options qui peut prendre les valeurs suivantes primary/success/info/warning/danger
buttonsG =[]
buttonsG.append(widgets.Button(description='free', button_style='success', layout=button_layout))
buttonsG.append(widgets.Button(description='blocked', button_style='danger', layout=button_layout))
for i in range(2,len(filenames)):
    if filenames[i] != 'vitesses' and filenames[i] != 'directions':
        buttonsG.append(widgets.Button(description=filenames[i], button_style='warning', layout=button_layout))

#vitesses
buttonsV =[]
for i in range(len(vitesses)):
    buttonsV.append(widgets.Button(description=vitesses[i], button_style='warning', layout=button_layout))

#directions
buttonsD =[]
for i in range(len(directions)):
    buttonsD.append(widgets.Button(description=directions[i], button_style='warning', layout=button_layout))
  

#----- COMPTEUR D'ELEMENTS -----
# Ces compteurs permettent de connaitre le nombre de photos pris dans chaque dossier
compteurs = {}
compteurs_cropped = {}
for directory in detectdirs:
    compteurs[directory.replace("dataset/","")] = widgets.IntText(layout=button_layout, value=len(os.listdir(directory)))

for directory in detectdirs_cropped:
    compteurs_cropped[directory.replace("dataset_cropped/","")] = widgets.IntText(layout=button_layout, value=len(os.listdir(directory)))


### Création des évènements lorsqu'on clique sur un bouton

In [ ]:
# ----- Evenement on click -----
from uuid import uuid1

#Fonction d'enregistrement
def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)
        
def refresh_compteur():
    global compteurs, compteurs_cropped
    for name in compteurs:
        compteurs[name].value = len(os.listdir('dataset/'+name))
    for name in compteurs_cropped:
        compteurs_cropped[name].value = len(os.listdir('dataset_cropped/'+name))
        
# Fonction qui va permettre d'établir le chemin d'accès pour enregistrer une photo en fonction du bouton sur lequel on a cliqué
def save(i):    
    if i<7 and i>3:
        save_snapshot('dataset/vitesses')
        save_snapshot('dataset_cropped/'+buttonsV[i-4].description)
    elif i>6:
        save_snapshot('dataset/directions')
        save_snapshot('dataset_cropped/'+ buttonsD[i-7].description)
    else :
        save_snapshot('dataset/'+buttonsG[i].description)
    refresh_compteur()
    return
    
# Création des évènements
# Le choix de l'utilisation des indices pour identifier les différents boutons n'est pas nécessairement très judicieux, il pourrait être interessant d'utiliser un dictionnaire ici
buttonsG[0].on_click(lambda x: save(0)) #free
buttonsG[1].on_click(lambda x: save(1)) #block
buttonsG[2].on_click(lambda x: save(2)) #panneau stop
buttonsG[3].on_click(lambda x: save(3)) #pieton

buttonsV[0].on_click(lambda x: save(4)) #vitesse 1
buttonsV[1].on_click(lambda x: save(5)) #vitesse 2
buttonsV[2].on_click(lambda x: save(6)) #vitesse 3

buttonsD[0].on_click(lambda x: save(7)) #direction droite
buttonsD[1].on_click(lambda x: save(8)) #direction gauche

### Affichage des boutons   
Vous pouvez alors dès à présent utiliser les boutons pour enregistrer des images

In [ ]:
for i in range(len(buttonsG)):
    display(widgets.HBox([compteurs[buttonsG[i].description], buttonsG[i]]))
    
for i in range(len(vitesses)):
    display(widgets.HBox([compteurs_cropped[vitesses[i]], buttonsV[i]]))

for i in range(len(directions)):
    display(widgets.HBox([compteurs_cropped[directions[i]], buttonsD[i]]))    

    
display(image)

### Arret de la caméra  
<font color='red'>ATTENTION !</font> vous devez éteindre la caméra avec le code suivant, avant de lancer un autre code au risque de ne pas pouvoir réutiliser la caméra par la suite sans devoir redémarrer le robot.

In [ ]:
camera.stop()

# Fonctions permettant de recadrer les images

### Fonction de détourage  
Après avoir testé l'utilisation du modèle avec des images "brutes" des panneaux, nous n'étions pas en mesure de différencier les panneaux de vitesses entre eux ni les panneaux de directions entre eux. Nous avons donc mis en place le code suivant. 

Cette fonciton permet de recadrer les photos enregistrées pour les panneaux de vitesses et les panneaux de directions, avec la structure des dossiers faite par le code précédent.  
On utilise dans cette fonction des filtres de couleur, rouge pour les panneaux de vitesses et bleu pour les panneaux de directions, pour ensuite détecter les contours de l'image.  
  
Vous trouverez davantages d'informations sur le lien suivant : https://stackoverflow.com/questions/30331944/finding-red-color-in-image-using-python-opencv


In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

def crop(path,sign):
    
    #image_read = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    img = cv2.imread(path)
    ## conversion en couleurs hsv
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    #filtrage
    # Set minimum and max HSV values to display
    if sign == 'vitesse':
        lower1 = np.array([0, 150, 50])
        upper1 = np.array([10, 255, 140])

        lower2 = np.array([169, 90, 50])
        upper2 = np.array([179, 255, 255])
        # Create HSV Image and threshold into a range.
        mask1 = cv2.inRange(hsv, lower1, upper1)
        mask2 = cv2.inRange(hsv, lower2, upper2)
        mask1 += mask2
    
    elif sign == 'direction':
        lower1 = np.array([100, 140, 50])
        upper1 = np.array([130, 255, 255])
        mask1 = cv2.inRange(hsv, lower1, upper1)
        
    target = cv2.bitwise_and(img,img, mask=mask1)



    # convert to RGB
    image = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)
    imge = cv2.bitwise_not(image)
    # convert to grayscale
    gray = cv2.cvtColor(imge, cv2.COLOR_RGB2GRAY)

    # create a binary thresholded image
    if sign == 'vitesse':
        _, binary = cv2.threshold(gray, 195, 255, cv2.THRESH_BINARY_INV)
    else: 
        _, binary = cv2.threshold(gray, 250, 250, cv2.THRESH_TOZERO_INV)
           
    #recherche du contour
    contours,hierarchy=cv2.findContours(binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)


    #Récupération du plus grand contour :
    indice = -1
    max = 0
    for i in range(len(contours)):
        long = len(contours[i])
        if long > max:
            indice = i
            max = long

    contour2 = [contours[indice]]

    # recadrage sur le plus grand contour:
    x = []
    y = []
    for couple in contour2[0]:
        x.append(couple[0][0])
        y.append(couple[0][1])

    xmax = -1
    xmin = 1000
    ymax = -1
    ymin = 1000
    
    #les conditions suivantes permettent de ne pas prendre en compte le contour de l'image entière qui peut apparaitre 
    for i in x:
        if 10 <i< 590 and i>xmax:
            xmax = i

        if 10<i< 590 and i<xmin:
            xmin = i
    for i in y:
        if 10 <i< 590 and i>ymax:
            ymax = i

        if 10<i< 590 and i<ymin:
            ymin = i   
    
    #Si l'image est trop petite on considère que c'est une erreur et qu'il n'y a pas de panneau sur l'image -> l'image sera alors supprimée
    if xmax-xmin < 25 or ymax-ymin<25:
        return "error: No image found"
    
    #on vient rogner l'image en rajoutant 10 pixels sur chaque côté pour avoir une vue un peu plus large de l'objet
    dim = (largeur_image,hauteur_image)
    
    return cv2.resize(img[ymin-10:ymax+10, xmin-10:xmax+10], dim, interpolation = cv2.INTER_AREA)

'''
# Code pour afficher l'image
cv2.imshow("cropped", crop_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-w1odqdyq because the default path (/home/jetbot/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Matplotlib is building the font cache; this may take a moment.


'\n# Code pour afficher l\'image\ncv2.imshow("cropped", crop_img)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n'

##### Code pour réaliser le recadrage de manière automatique à toutes les images enregistrées

In [ ]:
import os
 
path = 'dataset_cropped/'

for file in vitesses:
    pathV = path+file+'/'
    files = os.listdir(pathV)
    for name in files:
        if [name[i] for i in range(-4,0)] == ['.', 'j', 'p', 'g']: #certains fichiers cachés ne sont pas des images donc nous prenons la précaution de vérifier l'extension du fichier
            pathV = path+file+'/'
            pathV += name
            cropped_image = crop(pathV,'vitesse')
            if cropped_image == "error: No image found": #Dans le cas où le processus n'a pas trouvé de panneau, on supprime la photo
                os.remove(pathV)
            else:
                cv2.imwrite(pathV, cropped_image)

for file in directions:
    pathD = path+file+'/'
    files = os.listdir(pathD)
    for name in files:
        if [name[i] for i in range(-4,0)] == ['.', 'j', 'p', 'g']:
            pathD = path+file+'/'
            pathD += name
            cropped_image = crop(pathD,'direction')
            if cropped_image == "error: No image found":
                os.remove(pathD)
            else:
                cv2.imwrite(pathD, cropped_image)

#### Compléments  
Pour réaliser cette fonction de recadrage, nous avons utilisé le script suivant qui permet de voir en tant réel l'effet d'un filtre.

In [ ]:
import cv2
import sys
import numpy as np

def nothing(x):
    pass

# Load in image
image = cv2.imread('NOM_DE_L_IMAGE.jpg')

# Create a window
cv2.namedWindow('image')

# create trackbars for color change
cv2.createTrackbar('HMin','image',0,179,nothing) # Hue is from 0-179 for Opencv
cv2.createTrackbar('SMin','image',0,255,nothing)
cv2.createTrackbar('VMin','image',0,255,nothing)
cv2.createTrackbar('HMax','image',0,179,nothing)
cv2.createTrackbar('SMax','image',0,255,nothing)
cv2.createTrackbar('VMax','image',0,255,nothing)

# Set default value for MAX HSV trackbars.
cv2.setTrackbarPos('HMax', 'image', 179)
cv2.setTrackbarPos('SMax', 'image', 255)
cv2.setTrackbarPos('VMax', 'image', 255)

# Initialize to check if HSV min/max value changes
hMin = sMin = vMin = hMax = sMax = vMax = 0
phMin = psMin = pvMin = phMax = psMax = pvMax = 0

output = image
wait_time = 33

while(1):

    # get current positions of all trackbars
    hMin = cv2.getTrackbarPos('HMin','image')
    sMin = cv2.getTrackbarPos('SMin','image')
    vMin = cv2.getTrackbarPos('VMin','image')

    hMax = cv2.getTrackbarPos('HMax','image')
    sMax = cv2.getTrackbarPos('SMax','image')
    vMax = cv2.getTrackbarPos('VMax','image')

    # Set minimum and max HSV values to display
    lower = np.array([hMin, sMin, vMin])
    upper = np.array([hMax, sMax, vMax])

    # Create HSV Image and threshold into a range.
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    output = cv2.bitwise_and(image,image, mask= mask)

    # Print if there is a change in HSV value
    if( (phMin != hMin) | (psMin != sMin) | (pvMin != vMin) | (phMax != hMax) | (psMax != sMax) | (pvMax != vMax) ):
        print("(hMin = %d , sMin = %d, vMin = %d), (hMax = %d , sMax = %d, vMax = %d)" % (hMin , sMin , vMin, hMax, sMax , vMax))
        phMin = hMin
        psMin = sMin
        pvMin = vMin
        phMax = hMax
        psMax = sMax
        pvMax = vMax

    # Display output image
    cv2.imshow('image',output)

    # Wait longer to prevent freeze for videos.
    if cv2.waitKey(wait_time) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()